In [1]:
import transformers as tf
from tensorflow.keras.models import load_model

/home/shreyan/Documents/Github/Project-Dhi/dhienv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-18 18:15:58.279833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731933958.298677  125126 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731933958.303990  125126 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 18:15:58.322932: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in

In [2]:
model = load_model('linux_command_generator.keras')

2024-11-18 18:16:02.937717: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/home/shreyan/Documents/Github/Project-Dhi/dhienv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/shreyan/Documents/Github/Project-Dhi/dhienv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
from tensorflow.keras import Model

# Encoder setup
encoder_inputs = model.input[0]  # Input for the encoder
encoder_outputs, state_h, state_c = model.layers[4].output  # Outputs from the encoder LSTM
encoder_model = Model(encoder_inputs, [state_h, state_c])  # Define the encoder model

In [4]:
from tensorflow.keras.layers import Input, LSTM
from tensorflow.keras.models import Model

# Define decoder input sequence
decoder_inputs = Input(shape=(None,))  # Placeholder for decoder input sequences
decoder_embedding = model.layers[3](decoder_inputs)  # Embedding layer for decoder

# Retrieve LSTM units
units = model.layers[5].units

# Placeholders for decoder LSTM states
decoder_state_input_h = Input(shape=(units,))
decoder_state_input_c = Input(shape=(units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Recreate the decoder LSTM layer for inference
decoder_lstm_inference = LSTM(
    units,
    return_sequences=True,
    return_state=True,
    name="decoder_lstm_inference",
)
decoder_outputs, state_h, state_c = decoder_lstm_inference(
    decoder_embedding, initial_state=decoder_states_inputs
)

# Dense layer for output
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(decoder_outputs)

# Define the decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,  # Inputs for decoder sequence and states
    [decoder_outputs, state_h, state_c],  # Outputs: predictions and new states
)


In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_command(input_text, max_length=50):
    # Tokenize and pad the input
    input_seq = pad_sequences(
        input_tokeniser.texts_to_sequences([input_text]),
        maxlen=input_sequences.shape[1],  # Ensure padding matches the trained input size
        padding='post'
    )

    # Encode the input
    states_value = encoder_model.predict(input_seq)

    # Start the decoding process
    target_seq = pad_sequences([[output_tokeniser.word_index['<start>']]], maxlen=1)
    decoded_sentence = ''
    
    for _ in range(max_length):
        # Predict the next token
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # Get the most likely token
        sampled_token_index = output_tokens.argmax(axis=-1)[0, -1]
        sampled_word = output_tokeniser.index_word[sampled_token_index]
        
        # Break if the end token is reached
        if sampled_word == '<end>':
            break

        decoded_sentence += ' ' + sampled_word
        
        # Update the target sequence
        target_seq = pad_sequences([[sampled_token_index]], maxlen=1, padding='post')

        # Update states
        states_value = [h, c]
    
    return decoded_sentence.strip()


In [6]:
import pickle

# Load tokenisers
with open('input_tokeniser.pkl', 'rb') as handle:
    input_tokeniser = pickle.load(handle)

with open('output_tokeniser.pkl', 'rb') as handle:
    output_tokeniser = pickle.load(handle)

In [7]:
# Call the function
input_query = "(BSD specific) Display process information twice, waiting one second between each, filtering out the header line."
generated_command = generate_command(input_query)
print(f"Generated Command: {generated_command}")


NameError: name 'input_sequences' is not defined